# Put file metadata in database

In [1]:
import json
import numpy as np
import os

from db import d_models
from db import make_session, empty_tables

import CONFIG as C
import LOCAL as L
import PARAMS as P

In [2]:
def write_trials_from_file_system():
    """
    Look in the file system and extract paths, etc., of all the data files and store them
    in the database.
    """
    session = make_session()
    empty_tables(session, d_models.Trial)

    # get directories for different experiments

    for expt in next(os.walk(L.DATA_ROOT))[1]:
        print('LOADING DATA FILES FROM EXPERIMENT: "{}".'.format(expt))

        # get all of the bottom level directories (which contain the data files)
        data_dirs = []

        for root, dirs, files in os.walk(os.path.join(L.DATA_ROOT, expt)):
            if not dirs: data_dirs.append(root)

        # store trial for each data directory in database
        data_ctr = 0

        for data_dir in data_dirs:
            
            if os.path.basename(data_dir).startswith('~'):
                # skip if trial directory is marked with ~
                print('SKIPPING TRIAL: "{}"'.format(os.path.basename(data_dir)))
                continue
                
            print('TRIAL: "{}"'.format(os.path.basename(data_dir)))

            trial = d_models.Trial()

            trial.name = os.path.basename(data_dir)
            trial.expt = expt
            trial.fly = '.'.join(trial.name.split('.')[:2])
            trial.number = int(trial.name.split('.')[2])

            trial.path = os.path.relpath(data_dir, L.DATA_ROOT)

            # get names of all files in trial directory
            files = next(os.walk(data_dir))[2]

            # if asensory experiment
            if expt in C.EXPTS_ASENSORY:
                
                try:
                    f_behav = [
                        f for f in files if f.endswith(C.FILE_ENDINGS_ASENSORY['BEHAV'])][0]
                except:
                    raise Exception(
                        'Could not find BEHAV file in dir: "{}"'.format(data_dir))
                
                try:
                    f_gcamp = [
                        f for f in files if f.endswith(C.FILE_ENDINGS_ASENSORY['GCAMP'])][0]
                except:
                    raise Exception(
                        'Could not find GCAMP file in dir: "{}"'.format(data_dir))
                    
                f_t_gcamp = None
                f_light = None
                f_air = None
                f_odor_binary = None
                f_odor_pid = None
                details = None
            
            # if sensory experiment
            else:
                
                try:
                    f_behav = [
                        f for f in files if f.endswith(C.FILE_ENDINGS_SENSORY['BEHAV'])][0]
                except:
                    raise Exception(
                        'Could not find BEHAV file in dir: "{}"'.format(data_dir))

                try:
                    f_gcamp = [
                        f for f in files if f.endswith(C.FILE_ENDINGS_SENSORY['GCAMP'])][0]
                except:
                    raise Exception(
                        ('Could not find GCAMP file in dir: "{}"'.format(data_dir)))

                try:
                    f_t_gcamp = [
                        f for f in files if f.endswith(C.FILE_ENDINGS_SENSORY['T_GCAMP'])][0]
                except:
                    raise Exception(
                        ('Could not find T_GCAMP file in dir: "{}"'.format(data_dir)))

                try:
                    f_light = [
                        f for f in files if f.endswith(C.FILE_ENDINGS_SENSORY['LIGHT'])][0]
                except:
                    raise Exception(
                        ('Could not find LIGHT file in dir: "{}"'.format(data_dir)))

                try:
                    f_air = [
                        f for f in files if f.endswith(C.FILE_ENDINGS_SENSORY['AIR'])][0]
                except:
                    f_air = None

                # see if there is an odor_binary file
                try:
                    f_odor_binary = [
                        f for f in files if f.endswith(C.FILE_ENDINGS_SENSORY['ODOR_BINARY'])][0]
                    print('  odor_binary file found.')
                except:
                    f_odor_binary = None

                # see if there is an odor_pid file
                try:
                    f_odor_pid = [
                        f for f in files if f.endswith(C.FILE_ENDINGS_SENSORY['ODOR_PID'])][0]
                    print('  odor_pid file found.')
                except:
                    f_odor_pid = None

                # see if there is a details file
                if 'details.json' in files:
                    path = os.path.join(data_dir, 'details.json')
                    with open(path) as f: details = json.load(f)
                else:
                    details = {}

            trial.f_behav = f_behav
            trial.f_gcamp = f_gcamp
            trial.f_t_gcamp = f_t_gcamp
            trial.f_light = f_light
            trial.f_air = f_air
            trial.f_odor_binary = f_odor_binary
            trial.f_odor_pid = f_odor_pid
            
            trial.pfx_clean = C.PFX_CLEAN
            
            trial.details = details

            session.add(trial)

            data_ctr += 1

        session.commit()

        print(('{}/{} TRIALS LOADED FOR EXPERIMENT: '
               '"{}".'.format(data_ctr, len(data_dirs), expt)))

    print('Complete.')
    
write_trials_from_file_system()

LOADING DATA FILES FROM EXPERIMENT: "asensory_10_hz".
TRIAL: "20160524.Fly1.1"
TRIAL: "20160524.Fly2.1"
TRIAL: "20160524.Fly2.2"
TRIAL: "20160524.Fly2.3"
TRIAL: "20160524.Fly3.1"
TRIAL: "20160524.Fly4.1"
TRIAL: "20160525.Fly1.1"
TRIAL: "20160525.Fly1.2"
TRIAL: "20160525.Fly1.3"
TRIAL: "20160525.Fly2.1"
TRIAL: "20160525.Fly2.2"
TRIAL: "20160525.Fly3.1"
TRIAL: "20160525.Fly3.2"
TRIAL: "20160525.Fly4.1"
TRIAL: "20160525.Fly4.2"
TRIAL: "20160525.Fly5.1"
TRIAL: "20160525.Fly5.2"
TRIAL: "20160525.Fly5.3"
TRIAL: "20160525.Fly6.1"
TRIAL: "20160525.Fly6.2"
TRIAL: "20160526.Fly1.1"
TRIAL: "20160526.Fly2.1"
TRIAL: "20160531.Fly2.1"
TRIAL: "20160531.Fly2.2"
TRIAL: "20160531.Fly2.3"
TRIAL: "20160601.Fly1.2"
TRIAL: "20160601.Fly2.1"
TRIAL: "20160601.Fly4.1"
TRIAL: "20160614.Fly1.1"
TRIAL: "20160614.Fly1.2"
TRIAL: "20160614.Fly1.3"
TRIAL: "20160614.Fly1.4"
TRIAL: "20160614.Fly2.1"
TRIAL: "20160614.Fly2.2"
TRIAL: "20160614.Fly2.3"
TRIAL: "20160614.Fly3.1"
TRIAL: "20160614.Fly3.2"
TRIAL: "20160614.Fly3

# Upload hand-identified walking-speed thresholds to database

These are used to label each timepoint in a trial as belonging to either a "paused", "walking", or "ambiguous" state. They were determined by hand on a trial-by-trial basis and are stored in PARAMS.py.

def write_speed_thresholds_from_params():
    session = make_session()
    
    for name, th in P.SPEED_THRESHOLDS.items():
        
        trial = session.query(d_models.Trial).filter_by(name=name).first()
        trial.walking_threshold = th
        
        session.add(trial)
        session.commit()
        
    for trial in session.query(d_models.Trial).all():
        
        if trial.walking_threshold:
            print('Walking speed threshold for trial "{}" ='.format(trial.name))
            print(trial.walking_threshold)
        
    session.close()
        
write_speed_thresholds_from_params()